Урок 5. Spark on scala
Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year

"1984", "George Orwell", "Science Fiction", 5000, 1949

"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954

"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960

"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951

"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=deb250ef9bb1333f81da0be23528a6c69e84e32ed51b511066826928fba170df
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [22]:
# Импорт требуемых модулей
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

In [23]:
data = [("1984", "George Orwell", "Science Fiction", 5000, 1949),
("The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954),
("To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960),
("The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951),
("The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925)
]

schema = ['title', 'author', 'genre', 'sales', 'year']

In [24]:
# Установка сессии spark
spark = SparkSession.builder.appName('CreateCSV').getOrCreate()

# Создаем dataframe
df = spark.createDataFrame(data=data, schema = schema)
df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [25]:
# Записываем файл в папку 'data' на GoogleColab

df.repartition(1).write.csv('data', mode="overwrite", header=True)
spark.stop()

In [27]:
# Установка сессии spark
spark = SparkSession.builder.appName('HW5').getOrCreate()

# Читаем данные из файла
df = spark.read.format("csv").option("header",True) \
     .load(f"/content/data/{[f for f in os.listdir('/content/data/') if f[-4:] == '.csv'][0]}")

# Конвертируем данные string -> int
df = df.withColumn('sales',df['sales'].cast('int'))
df = df.withColumn('year',df['year'].cast('int'))

df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [29]:
# Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров
df_more_3000 = df.filter(F.col('sales') > 3000)
df_more_3000.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [32]:
# Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
df_genre_sales1 = df_more_3000.groupBy('genre').sum('sales') \
             .withColumnRenamed('sum(sales)', 'total_sales')
df_genre_sales1.show()

+---------------+-----------+
|          genre|total_sales|
+---------------+-----------+
|Southern Gothic|       4000|
|          Novel|       4500|
|Science Fiction|       5000|
+---------------+-----------+



In [39]:
#Не очень понятно какие именно данные группировать, сделаю и для первоначального датасета
df_genre_sales2  = df.groupBy('genre').sum('sales') \
            .withColumnRenamed('sum(sales)', 'total_sales')
df_genre_sales2.show()

+---------------+-----------+
|          genre|total_sales|
+---------------+-----------+
|Southern Gothic|       4000|
|          Novel|       6500|
|        Fantasy|       3000|
|Science Fiction|       5000|
+---------------+-----------+



In [41]:
#  Отсортируйте данные по общему объему продаж в порядке убывания
df_sort1 = df_genre_sales1.sort(F.col('total_sales').desc())
df_sort1.show()

+---------------+-----------+
|          genre|total_sales|
+---------------+-----------+
|Science Fiction|       5000|
|          Novel|       4500|
|Southern Gothic|       4000|
+---------------+-----------+



In [44]:
#  Отсортируйте данные по общему объему продаж в порядке убывания
df_sort2 = df_genre_sales2.sort(F.col('total_sales').desc())
df_sort2.show()

+---------------+-----------+
|          genre|total_sales|
+---------------+-----------+
|          Novel|       6500|
|Science Fiction|       5000|
|Southern Gothic|       4000|
|        Fantasy|       3000|
+---------------+-----------+



In [45]:
spark.stop()